In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [40]:
df = pd.read_csv("preprocessed_eeg_features.csv")
print(df.shape)

(30, 664)


In [41]:
df['internet_addiction_level'] = df['internet_addiction_level'].replace({'Moderate level': 'Severe Level'})

In [42]:
from sklearn.preprocessing import LabelEncoder

le_gender = LabelEncoder()
le_internet_addiction = LabelEncoder()

df['gender_encoded'] = le_gender.fit_transform(df['gender'])
df['internet_addiction_level_encoded'] = le_internet_addiction.fit_transform(df['internet_addiction_level'])


In [43]:
X = df.drop(columns = ['gender', 'internet_addiction_level', 'personality', 'internet_addiction_level_encoded'])
y = df['internet_addiction_level_encoded']

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Training size:", len(X_train))
print("Test size:", len(X_test))

Training size: 24
Test size: 6


In [45]:
X_test

task1_attention_mean  task1_attention_std  task1_meditation_mean  \
8              28.966667            23.323784              68.350000   
26             88.483333             6.414805              71.750000   
0              52.883333            17.364803              61.866667   
20             62.166667            17.948228              72.450000   
12             63.266667            20.735530              71.566667   
14             64.266667            12.443026              60.966667   

    task1_meditation_std  task1_LowAlpha_mean  task1_LowAlpha_std  \
8              19.101331         73833.733333        51581.965970   
26             19.148216         23553.700000        18329.807417   
0               9.947306         33254.633333        28935.250292   
20             16.934605         14308.300000        17958.097102   
12             18.945859         16482.750000        15773.218779   
14             17.856807         33006.116667        30110.608041   

    task1_LowAlpha_kurtosis  task1_LowAlpha_skewness  task1_LowAlpha_range  \
8                  2.061505                 1.356954                259187   
26                 0.325923                 0.957951                 82099   
0                  2.126606                 1.465773                133649   
20                21.690776                 4.161194                123641   
12                13.184229                 3.028866                102223   
14                 0.895741                 1.244914                121428   

    task1_LowAlpha_bandpower_sum  ...  task11_LowGamma_spectral_entropy  \
8                        4430024  ...                          3.553326   
26                       1413222  ...                          2.589343   
0                        1995278  ...                          1.886062   
20                        858498  ...                          1.932441   
12                        988965  ...                          2.010206   
14                       1980367  ...                          2.043026   

    task11_MiddleGamma_mean  task11_MiddleGamma_std  \
8              10672.866667            18607.258988   
26              6792.816667             6692.739816   
0              12510.316667            10064.408364   
20              9838.083333             7203.596410   
12              8553.650000             7417.646497   
14              3313.000000             3007.605709   

    task11_MiddleGamma_kurtosis  task11_MiddleGamma_skewness  \
8                     15.206576                     3.846065   
26                     1.808791                     1.450244   
0                      5.744636                     2.010266   
20                     3.199001                     1.524321   
12                     3.288384                     1.705361   
14                    15.157205                     3.228067   

    task11_MiddleGamma_range  task11_MiddleGamma_bandpower_sum  \
8                     107803                            640372   
26                     30852                            407569   
0                      57835                            750619   
20                     38405                            590285   
12                     38053                            513219   
14                     20050                            198780   

    task11_MiddleGamma_spectral_entropy  memory_words_correct  gender_encoded  
8                              3.438652                     7               1  
26                             2.460088                    12               0  
0                              2.123307                     9               1  
20                             1.924567                     6               1  
12                             2.224665                     9               0  
14                             2.388962                    10               0  

[6 rows x 662 columns]

In [46]:
y_test

8     2
26    0
0     1
20    0
12    0
14    0
Name: internet_addiction_level_encoded, dtype: int32

In [47]:
le_internet_addiction.__dict__

{'classes_': array(['Mid Level', 'Normal Level', 'Severe Level'], dtype=object)}

In [48]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [49]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [50]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(k_neighbors = 2, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_pca, y_train)

print("After SMOTE - Training size:", len(X_train_resampled))
print("Class distribution:\n", pd.Series(y_train_resampled).value_counts())

After SMOTE - Training size: 42
Class distribution:
 internet_addiction_level_encoded
0    14
2    14
1    14
Name: count, dtype: int64


In [51]:
def augment_features_and_labels(X, y, n_augmentations=5, noise_level=0.01, scale_range=(0.9, 1.1), max_shift=2):
    augmented_X = []
    augmented_y = []

    for _ in range(n_augmentations):
        X_aug = X.copy()

        operation = np.random.choice(['noise', 'scale', 'rolling'])

        if operation == 'noise':
            noise = np.random.normal(0, noise_level, X_aug.shape)
            X_aug += noise

        elif operation == 'scale':
            scale_factors = np.random.uniform(scale_range[0], scale_range[1], X_aug.shape[1])
            X_aug *= scale_factors

        elif operation == 'rolling':
            for col_idx in range(X.shape[1]):
                shift = np.random.randint(-max_shift, max_shift + 1)
                X_aug[:, col_idx] = np.roll(X[:, col_idx], shift)

        augmented_X.append(X_aug)
        augmented_y.append(y)

    X_augmented = np.vstack(augmented_X)
    y_augmented = np.concatenate(augmented_y)

    return X_augmented, y_augmented

In [52]:
X_train_aug, y_train_aug = augment_features_and_labels(X_train_resampled, y_train_resampled, n_augmentations=5)

print("Resampled training size:", X_train_resampled.shape)
print("Augmented training size:", X_train_aug.shape)

Resampled training size: (42, 19)
Augmented training size: (210, 19)


## ML Models

In [53]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import seaborn as sns

In [54]:
def evaluate_model_with_cv(model, param_grid, X_real, y_real, X_aug, y_aug, model_name="Model"):
    # Step 1: Grid Search on Augmented Data
    grid = GridSearchCV(model, param_grid, cv=StratifiedKFold(n_splits=5), scoring='f1_macro', n_jobs=-1)
    grid.fit(X_aug, y_aug)

    best_model = grid.best_estimator_
    print(f"\n---- {model_name} Best Params ----")
    print(grid.best_params_)

    # Step 2: Cross-Validation on Real (non-augmented) data
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    all_preds = []
    all_true = []

    y_real = np.array(y_real)

    for train_idx, val_idx in skf.split(X_real, y_real):
        X_tr, X_val = X_real[train_idx], X_real[val_idx]
        y_tr, y_val = y_real[train_idx], y_real[val_idx]

        best_model.fit(X_tr, y_tr)
        preds = best_model.predict(X_val)

        all_preds.extend(preds)
        all_true.extend(y_val)

    print(f"\n---- {model_name} Cross-Validation Report ----")
    print(classification_report(all_true, all_preds))

    cm = confusion_matrix(all_true, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"{model_name} - Cross-Validation Confusion Matrix")
    plt.show()

    return best_model

In [55]:
def evaluate_model(y_true, y_pred, model_name):
    print(classification_report(y_true, y_pred))

    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"{model_name} - Test Set Confusion Matrix")
    plt.show()

### SVM

In [56]:
from sklearn.svm import SVC

svm_param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}

best_svm = evaluate_model_with_cv(
    model=SVC(probability=True),
    param_grid=svm_param_grid,
    X_real=X_train_resampled,      # real (non-augmented) training set
    y_real=y_train_resampled,
    X_aug=X_train_aug,   # augmented training set
    y_aug=y_train_aug,
    model_name="SVM"
)



---- SVM Best Params ----
{'C': 0.1, 'kernel': 'linear'}

---- SVM Cross-Validation Report ----
              precision    recall  f1-score   support

           0       0.83      0.36      0.50        14
           1       0.82      1.00      0.90        14
           2       0.68      0.93      0.79        14

    accuracy                           0.76        42
   macro avg       0.78      0.76      0.73        42
weighted avg       0.78      0.76      0.73        42



### Random Forest Classification

In [57]:
rf_param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, None]
}

best_rf = evaluate_model_with_cv(
    model=RandomForestClassifier(random_state=42),
    param_grid=rf_param_grid,
    X_real=X_train_resampled,      # real (resampled, non-augmented)
    y_real=y_train_resampled,
    X_aug=X_train_aug,             # full augmented set
    y_aug=y_train_aug,
    model_name="Random Forest"
)


---- Random Forest Best Params ----
{'max_depth': 5, 'n_estimators': 50}

---- Random Forest Cross-Validation Report ----
              precision    recall  f1-score   support

           0       0.67      0.57      0.62        14
           1       0.82      1.00      0.90        14
           2       0.77      0.71      0.74        14

    accuracy                           0.76        42
   macro avg       0.75      0.76      0.75        42
weighted avg       0.75      0.76      0.75        42



### XGBoost

In [58]:
xgb_param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1]
}

best_xgb = evaluate_model_with_cv(
    model=XGBClassifier(eval_metric='mlogloss', random_state=42),
    param_grid=xgb_param_grid,
    X_real=X_train_resampled,
    y_real=y_train_resampled,
    X_aug=X_train_aug,
    y_aug=y_train_aug,
    model_name="XGBoost"
)


---- XGBoost Best Params ----
{'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}

---- XGBoost Cross-Validation Report ----
              precision    recall  f1-score   support

           0       0.64      0.50      0.56        14
           1       0.87      0.93      0.90        14
           2       0.62      0.71      0.67        14

    accuracy                           0.71        42
   macro avg       0.71      0.71      0.71        42
weighted avg       0.71      0.71      0.71        42



In [59]:
import joblib
joblib.dump(best_svm, f'best_model_svm.pkl')
joblib.dump(best_rf, "best_rf.pkl")
joblib.dump(best_xgb, "best_xgb.pkl")

['best_xgb.pkl']

In [60]:
joblib.dump(le_gender, "inference/le_genger.pkl")
joblib.dump(le_internet_addiction, "inference/le_internet_addiction.pkl")
joblib.dump(pca, "inference/pca.pkl")
joblib.dump(sc, 'inference/sc.pkl')

['inference/sc.pkl']